## Prompting and Chain-of-thought

> Algunas ideas se tomaron de este [notebook](https://github.com/aws-samples/amazon-nova-samples/blob/main/multimodal-understanding/workshop/2-prompting-best-practices.ipynb)

In [ ]:
import boto3
import json
from typing import Dict, List, Any
from IPython.display import display, Markdown

In [ ]:
# SDK & Client. See region!
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')
bedrock_client = boto3.client('bedrock', region_name='us-west-2')

In [ ]:
def call_nova(
    model: str,
    messages: List[Dict],
    system_message: str = "",
    max_tokens: int = 512,
    temp: float = 0.7,
    top_p: float = 0.99,
    top_k: int = 20,
    stop_sequences: List[str] = [],
) -> Any:
    """
    Call Nova model through AWS Bedrock for text generation or streaming.
    """
    try:
        # Prepare request body
        request_body = {
            "messages": messages,
            "system": [{"text": system_message}],
            "inferenceConfig": {
                "max_new_tokens": max_tokens,
                "top_p": top_p,
                "top_k": top_k,
                "temperature": temp,
                "stopSequences": stop_sequences,
            }
        }
        response = bedrock_runtime.invoke_model(
            modelId=model, 
            body=json.dumps(request_body)
        )
        model_response = json.loads(response["body"].read())
        
        return model_response, model_response["output"]["message"]["content"][0]["text"]
    except Exception as e:
        raise Exception(f"Error calling Nova model: {str(e)}")
    
def print_output(content_text):
    display(Markdown(content_text))

In [ ]:
# Elige tu modelo, acorde a tu región: https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html 
# - Intenta con otros modelos de Amazon: us.amazon.nova-micro-v1:0, us.amazon.nova-lite-v1:0, us.amazon.nova-pro-v1:0
model_id = "us.amazon.nova-micro-v1:0"

In [ ]:
no_shot = """Your task is to Classify the following texts into the appropriate sentiment classes. The categories to classify are:

Sentiment Classes:
- Positive
- Negative
- Neutral

Query:
Input: The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline.

"""

messages = [{"role": "user", "content": [{"text": no_shot}]}]

model_response, content_text = call_nova(model_id, messages)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

#### Añadamos "a few shot" al prompt!
> Intenta cambiar el prompt; e.g. "The movie is only good if you need a nap in the theatre."

In [ ]:
four_shot = """Your task is to Classify the following texts into the appropriate sentiment classes. The categories to classify are:

Sentiment Classes:
- Positive
- Negative
- Neutral

Please refer to some examples mentioned below.

## Examples
### Example 1
Input: The movie was crazy good! I loved it
Output: Positive
Explaination: The text said "good" and "loved" so its positive

### Example 2
Input: The movie was scary and I got scared!
Output: Neutral
Explaination: The text said "scary" and "scared" which can be both positive and negative depending on people who like scary movies or one who hate

### Example 3
Input: The movie was pathetic not worth the time or money!
Output: Negative
Explaination: The text said "pathetic" and "not worth" which is negative sentiment

### Example 4
Input: The movie had some plots which were interesting and great while there were some gaps which needed more drama!
Output: Neutral
Explaination: The text said "interesting and great" and "some gaps" making it a mixed opinion hence neutral

Query:
Input: Just watch a different movie. Try maybe one from Sandra Bullock 🤬
"""


messages = [{"role": "user", "content": [{"text": four_shot}]}]

model_response, content_text = call_nova(model_id, messages)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

## Chain of Thought (CoT)

In [ ]:
# User Input
no_cot = """In a room of 100 people, 99 percent are left-handed. 
How many left-handed people have to leave the room to bring that percentage down to 98 percent?
"""

# Format message
messages = [{"role": "user", "content": [{"text": no_cot}]}]

# Call Nova
model_response, content_text = call_nova(model_id, messages, max_tokens=1024)

# Show response
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)

#### Intentemos otro modelo

In [ ]:
# Elige tu modelo, acorde a tu región: https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html 
# - Intenta con otros modelos: anthropic.claude-3-5-haiku-20241022-v1:0, anthropic.claude-3-haiku-20240307-v1:0
anthropic_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Call Bedrock Runtime
response = bedrock_runtime.converse(
      modelId=anthropic_model_id,
      system=[{
        "text": "Read the following text and answer the question."
      }],
      messages=[{
        "role": "user",
        "content": [{"text": no_cot}]
      }],
      inferenceConfig={
          "maxTokens": 4096,
          "temperature": 0.5
      },
    )

# Print the response
response_text = response['output']['message']['content'][0]['text']
print(response_text)

### Ahora usemos CoT

In [ ]:
# User question
user_question = "In a room of 100 people, 99 percent are left-handed. How many left-handed people have to leave the room to bring that percentage down to 98 percent?"

# Guided Chain of Thought
guided_cot = f"""For tasks requiring reasoning or math, use the Chain-of-Thought methodology to explain your step-by-step calculations or logic

Solve the questions systematically, following these guidelines:

You are using a Chain of Thought (CoT) approach with reflection to answer queries. Follow these steps:

1. Think through the problem step by step within the <thinking> tags.
2. Reflect on your thinking to check for any errors or improvements within the <reflection> tags.
3. Make any necessary adjustments based on your reflection.
4. Show the steps in Markdown format you took to solve the problem, including reasoning, reflection and calculations.

Now, apply these steps to solve:
{user_question}

Show your work at each step, explaining your reasoning.
"""

messages = [{"role": "user", "content": [{"text": guided_cot}]}]
model_response, content_text = call_nova(model_id, messages, max_tokens=2048)

# Print the response
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)

### Con otro modelo:

In [ ]:
# - Intenta con otros modelos: anthropic.claude-3-5-haiku-20241022-v1:0, anthropic.claude-3-haiku-20240307-v1:0, us.deepseek.r1-v1:0, us.meta.llama3-2-3b-instruct-v1:0
anthropic_model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"

response = bedrock_runtime.converse(
      modelId=anthropic_model_id,
      system=[{
        "text": "Read the following Chain-of-Thought (CoT) reasoning process and answer the question."
      }],
      messages=[{
        "role": "user",
        "content": [{"text": guided_cot}]
      }],
      inferenceConfig={
          "maxTokens": 4096,
          "temperature": 0.5
      },
    )

# Print the response
response_text = response['output']['message']['content'][0]['text']
print_output(response_text)

### Otros ejemplos

In [ ]:
# Elige tu modelo, acorde a tu región: https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html 
# - Intenta con otros modelos de Amazon: us.amazon.nova-micro-v1:0, us.amazon.nova-lite-v1:0, us.amazon.nova-pro-v1:0
amzn_model_id = "us.amazon.nova-micro-v1:0"

### Sin CoT

In [ ]:
no_cot = """You are a project manager for a small software development team tasked with launching a new app feature.
You want to streamline the development process and ensure timely delivery. Draft a project plan
"""
messages = [{"role": "user", "content": [{"text": no_cot}]}]

model_response, content_text = call_nova(amzn_model_id, messages, max_tokens=1024)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

### Con CoT

In [ ]:
guided_cot = """You are a project manager for a small software development team tasked with launching a new app feature.
You want to streamline the development process and ensure timely delivery.
Your task is to draft a project plan.

But first do some thinking on how you want to structure and go through below questions before starting the draft.
Please follow these steps:
1. Think about who the audience is (this is for CEOs, CTOs and other executives)
2. Think about what to start with
3. Think about what Challenges you want to solve with this app
4. Think about the Tasks that will be needed to be completed
5. Create Milestones
6. Monitor Progress and Optimize
Explain all your thinking in <thinking></thinking> XML Tags and then write the final copy of project plan for executives in <project_plan></project_plan> XML Tag.

Output Schema:
<thinking>
( thoughts to above questions)
</thinking>
<project_plan>
( project plan)
</project_plan>
"""
messages = [{"role": "user", "content": [{"text": guided_cot}]}]

model_response, content_text = call_nova(amzn_model_id, messages, max_tokens=2048)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

#### Confirma acceso a modelos, como son Sonnet 3.7

In [ ]:
# - Intenta con otros modelos: us.anthropic.claude-3-7-sonnet-20250219-v1:0, anthropic.claude-3-5-haiku-20241022-v1:0, anthropic.claude-3-haiku-20240307-v1:0, us.deepseek.r1-v1:0, us.meta.llama3-2-3b-instruct-v1:0
anthropic_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

response = bedrock_runtime.converse(
      modelId=anthropic_model_id,
      system=[{
        "text": "Read the following Chain-of-Thought (CoT) reasoning process and answer the question."
      }],
      messages=[{
        "role": "user",
        "content": [{"text": guided_cot}]
      }],
      inferenceConfig={
          "maxTokens": 4096,
          "temperature": 0.5
      },
    )

# Print the response
response_text = response['output']['message']['content'][0]['text']
print_output(response_text)